In [41]:
import pandas as pd
import json
import os 

In [81]:
# Читаем файл 
data = pd.read_csv('archive.zip', compression="zip")
print(data)

                id                 date  client_id  card_id   amount  \
0          7475327  2010-01-01 00:01:00       1556     2972  $-77.00   
1          7475328  2010-01-01 00:02:00        561     4575   $14.57   
2          7475329  2010-01-01 00:02:00       1129      102   $80.00   
3          7475331  2010-01-01 00:05:00        430     2860  $200.00   
4          7475332  2010-01-01 00:06:00        848     3915   $46.41   
...            ...                  ...        ...      ...      ...   
13305910  23761868  2019-10-31 23:56:00       1718     2379    $1.11   
13305911  23761869  2019-10-31 23:56:00       1766     2066   $12.80   
13305912  23761870  2019-10-31 23:57:00        199     1031   $40.44   
13305913  23761873  2019-10-31 23:58:00       1986     5443    $4.00   
13305914  23761874  2019-10-31 23:59:00        489     5697   $12.88   

                    use_chip  merchant_id merchant_city merchant_state  \
0          Swipe Transaction        59935        Beulah      

In [82]:
# a. Объем памяти, который занимает файл на диске
file_path = r"C:/Users/ipavl/Учеба_УРФУ/1_курс/1_семестр/ИД/Practice_6/archive.zip" 
file_info = os.stat(file_path)
file_size_bytes = file_info.st_size

# Conversion to kilobytes, megabytes, and gigabytes
file_size_kb = file_size_bytes / 1024
file_size_mb = file_size_kb / 1024

print(f"File Size (Bytes): {file_size_bytes} B")
print(f"File Size (KB): {file_size_kb:.2f} KB")
print(f"File Size (MB): {file_size_mb:.2f} MB")

File Size (Bytes): 322644658 B
File Size (KB): 315082.67 KB
File Size (MB): 307.70 MB


In [83]:
# b.Объем памяти, который занимает набор данных при загрузке в память
def mem_usage(pandas_obj):
    if isinstance(pandas_obj, pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else:  # предположим, что если это не датафрейм, то серия
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2  # преобразуем байты в мегабайты
    return "{:03.2f} MB".format(usage_mb)

mem_usage(data)

'4766.50 MB'

In [86]:
#c.	Вычислить для каждой колонки занимаемый объем памяти, долю от общего объема, а также выяснить тип данных
# Полученный набор данных отсортировать по занимаемому объему памяти. 
# Вывести в файл (json) данные по колонкам с пометкой, что это статистика по набору данных без применения оптимизаций.

def get_memory_stat_by_column(data):
    memory_usage_stat = data.memory_usage(deep=True)
    total_memory_usage_kb = int(memory_usage_stat.sum() // 1024)
    print(f"file in memory size = {total_memory_usage_kb:10} KB")
    column_stat = []
    for key in data.columns:
        column_stat.append({
            "column_name": key,
            "memory_abs": int(memory_usage_stat[key] // 1024), #Explicit int conversion
            "memory_per": round(memory_usage_stat[key] / memory_usage_stat.sum() * 100, 4),
            "dtype": str(data[key].dtype)
        })
    column_stat.sort(key=lambda x: x['memory_abs'], reverse=True)

    for column in column_stat:
        print(
            f"{column['column_name']:30}: {column['memory_abs']:10} KB: {column['memory_per']:10}% : {column['dtype']}")

    return {"total_memory_usage_kb": total_memory_usage_kb, "columns": column_stat}

result = get_memory_stat_by_column(data)
with open('result.json', 'w') as f:
    json.dump({"memory_usage": result}, f, indent=4)

file in memory size =    4880892 KB
date                          :     883595 KB:    18.1032% : object
use_chip                      :     854460 KB:    17.5062% : object
merchant_city                 :     748112 KB:    15.3274% : object
amount                        :     714092 KB:    14.6304% : object
merchant_state                :     634129 KB:    12.9921% : object
errors                        :     422786 KB:     8.6621% : object
id                            :     103952 KB:     2.1298% : int64
client_id                     :     103952 KB:     2.1298% : int64
card_id                       :     103952 KB:     2.1298% : int64
merchant_id                   :     103952 KB:     2.1298% : int64
zip                           :     103952 KB:     2.1298% : float64
mcc                           :     103952 KB:     2.1298% : int64


In [96]:
optimized_dataset = data.copy()

In [97]:
# 4.	Преобразовать все колонки с типом данных «object» в категориальные, если количество уникальных значений колонки составляет менее 50%.

def opt_obj(data):
    converted_obj = pd.DataFrame()
    dataset_obj = data.select_dtypes(include=['object']).copy()

    for col in dataset_obj.columns:
        num_unique_values = len(dataset_obj[col].unique())
        num_total_values = len(dataset_obj[col])
        if num_unique_values / num_total_values < 0.5:
            converted_obj.loc[:, col] = dataset_obj[col].astype('category')
        else:
            converted_obj.loc[:, col] = dataset_obj[col]

    print(mem_usage(dataset_obj))
    print(mem_usage(converted_obj))
    return converted_obj

converted_obj = opt_obj(data)
print(converted_obj)

4157.40 MB
582.22 MB
                         date   amount            use_chip merchant_city  \
0         2010-01-01 00:01:00  $-77.00   Swipe Transaction        Beulah   
1         2010-01-01 00:02:00   $14.57   Swipe Transaction    Bettendorf   
2         2010-01-01 00:02:00   $80.00   Swipe Transaction         Vista   
3         2010-01-01 00:05:00  $200.00   Swipe Transaction   Crown Point   
4         2010-01-01 00:06:00   $46.41   Swipe Transaction       Harwood   
...                       ...      ...                 ...           ...   
13305910  2019-10-31 23:56:00    $1.11    Chip Transaction   West Covina   
13305911  2019-10-31 23:56:00   $12.80  Online Transaction        ONLINE   
13305912  2019-10-31 23:57:00   $40.44   Swipe Transaction         Allen   
13305913  2019-10-31 23:58:00    $4.00    Chip Transaction     Daly City   
13305914  2019-10-31 23:59:00   $12.88    Chip Transaction    Greenbrier   

         merchant_state errors  
0                    ND    NaN  


In [98]:
# 5.	Провести понижающее преобразование типов «int» колонок 
def opt_int(data):
    dataset_int = data.select_dtypes(include=['int'])
    """
    downcast:
            - 'integer' or 'signed': smallest signed int dtype (min.: np.int8)
            - 'unsigned': smallest unsigned int dtype (min.: np.uint8)
            - 'float': smallest float dtype (min.: np.float32)
    """
    converted_int = dataset_int.apply(pd.to_numeric, downcast='unsigned')
    print(mem_usage(dataset_int))
    print(mem_usage(converted_int))
    #
    compare_ints = pd.concat([dataset_int.dtypes, converted_int.dtypes], axis=1)
    compare_ints.columns = ['before', 'after']
    compare_ints.apply(pd.Series.value_counts)
    print(compare_ints)

    return converted_int

converted_int = opt_int(data)
print(converted_int)

507.58 MB
177.65 MB
            before   after
id           int64  uint32
client_id    int64  uint16
card_id      int64  uint16
merchant_id  int64  uint32
mcc          int64  uint16
                id  client_id  card_id  merchant_id   mcc
0          7475327       1556     2972        59935  5499
1          7475328        561     4575        67570  5311
2          7475329       1129      102        27092  4829
3          7475331        430     2860        27092  4829
4          7475332        848     3915        13051  5813
...            ...        ...      ...          ...   ...
13305910  23761868       1718     2379        86438  5499
13305911  23761869       1766     2066        39261  5815
13305912  23761870        199     1031         2925  4900
13305913  23761873       1986     5443        46284  5411
13305914  23761874        489     5697        24658  5921

[13305915 rows x 5 columns]


In [99]:
# 6.	Провести понижающее преобразование типов «float» колонок
def opt_float(data):
    dataset_float = data.select_dtypes(include=['float'])
    converted_float = dataset_float.apply(pd.to_numeric, downcast='float')

    print(mem_usage(dataset_float))
    print(mem_usage(converted_float))

    compare_floats = pd.concat([dataset_float.dtypes, converted_float.dtypes], axis=1)
    compare_floats.columns = ['before', 'after']
    compare_floats.apply(pd.Series.value_counts)
    print(compare_floats)

    return converted_float

converted_float = opt_float(data)
print(converted_float)

101.52 MB
50.76 MB
      before    after
zip  float64  float32
              zip
0         58523.0
1         52722.0
2         92084.0
3         46307.0
4         20776.0
...           ...
13305910  91792.0
13305911      NaN
13305912  75002.0
13305913  94014.0
13305914  37073.0

[13305915 rows x 1 columns]


In [100]:
optimized_dataset[converted_obj.columns] = converted_obj
optimized_dataset[converted_int.columns] = converted_int
optimized_dataset[converted_float.columns] = converted_float

In [101]:
get_memory_stat_by_column(data)
print(mem_usage(data))
print(mem_usage(optimized_dataset))
optimized_dataset.info(memory_usage='deep')

file in memory size =    4880892 KB
date                          :     883595 KB:    18.1032% : object
use_chip                      :     854460 KB:    17.5062% : object
merchant_city                 :     748112 KB:    15.3274% : object
amount                        :     714092 KB:    14.6304% : object
merchant_state                :     634129 KB:    12.9921% : object
errors                        :     422786 KB:     8.6621% : object
id                            :     103952 KB:     2.1298% : int64
client_id                     :     103952 KB:     2.1298% : int64
card_id                       :     103952 KB:     2.1298% : int64
merchant_id                   :     103952 KB:     2.1298% : int64
zip                           :     103952 KB:     2.1298% : float64
mcc                           :     103952 KB:     2.1298% : int64
4766.50 MB
810.63 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column     

In [104]:
# 8.	Выбрать произвольно 10 колонок для дальнейшем работы, прописав преобразование 
# типов и загрузку только нужных данных на этапе чтения файла. 
# При этом стоит использовать чанки. 
# Сохраните полученный поднабор в отдельном файле.

need_column = dict()
column_names = ['date', 'client_id', 'card_id',
                 'amount', 'use_chip', 'merchant_id',
                 'merchant_city', 'merchant_state', 'zip', 'mcc']
opt_dtypes = optimized_dataset.dtypes
for key in data.columns:
    need_column[key] = opt_dtypes[key]
    print(f"{key}:{opt_dtypes[key]}")

with open("dtypes.json", mode="w") as file:
    dtype_json = need_column.copy()
    for key in dtype_json.keys():
        dtype_json[key] = str(dtype_json[key])

    json.dump(dtype_json, file)

id:uint32
date:category
client_id:uint16
card_id:uint16
amount:category
use_chip:category
merchant_id:uint32
merchant_city:category
merchant_state:category
zip:float32
mcc:uint16
errors:category


In [106]:
# Используя оптимизированный набор данных, построить пять-семь графиков 
# (включая разные типы: линейный, столбчатый, круговая диаграмма, корреляция и т.д.)
read_and_optimized = pd.read_csv('archive.zip', compression="zip", usecols=lambda x: x in column_names, dtype=need_column)
read_and_optimized.head()

,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc
0,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499
1,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311
2,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829
3,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829
4,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813
